In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/national-elections/ABIA_unsure.csv
/kaggle/input/national-elections/ABIA_notfound.csv
/kaggle/input/national-elections/ABIA_crosschecked.csv


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv
import os
import requests

from geopy.distance import geodesic
from scipy.spatial import distance_matrix

import warnings
warnings.filterwarnings("ignore")

In [30]:
crosschecked_df = pd.read_csv('/kaggle/input/national-elections/ABIA_crosschecked.csv')
notfound_df = pd.read_csv('/kaggle/input/national-elections/ABIA_notfound.csv')
unsure_df = pd.read_csv('/kaggle/input/national-elections/ABIA_unsure.csv')

# Combine the dataframes if necessary
combined_df = pd.concat([crosschecked_df, notfound_df, unsure_df], ignore_index=True)
combined_df.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File
0,ABIA,ABA NORTH,EZIAMA,01-01-01-001,RAILWAY QUARTERS - RAILWAY QUARTERS I,85,968,1,-1,1,1,0,0,UNKNOWN,7,56,25,1,https://docs.inecelectionresults.net/elections...
1,ABIA,ABA NORTH,EZIAMA,01-01-01-002,RAILWAY QUARTERS - RAILWAY QUARTERS II,90,750,1,-1,0,0,0,0,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...
2,ABIA,ABA NORTH,EZIAMA,01-01-01-003,RAILWAY QUARTERS - RAILWAY QUARTERS III,105,750,1,-1,0,0,0,0,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...
3,ABIA,ABA NORTH,EZIAMA,01-01-01-005,ABIA POLY - ABIA POLY I,138,750,1,-1,0,0,0,0,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...
4,ABIA,ABA NORTH,INDUSTRIAL AREA,01-01-02-012,LEVER BROTHERS GATE- LEVER BROTHERS GATE IV,71,774,1,-1,0,0,0,0,UNKNOWN,0,71,0,0,https://docs.inecelectionresults.net/elections...


In [20]:
# Shuffle the data
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Sample the data (for example, 10% of the data)
sampled_df = shuffled_df.sample(frac=0.1, random_state=42).reset_index(drop=True)

combined_df = sampled_df

In [31]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4062 entries, 0 to 4061
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   State                   4062 non-null   object
 1   LGA                     4062 non-null   object
 2   Ward                    4062 non-null   object
 3   PU-Code                 4062 non-null   object
 4   PU-Name                 4062 non-null   object
 5   Accredited_Voters       4062 non-null   int64 
 6   Registered_Voters       4062 non-null   int64 
 7   Results_Found           4062 non-null   int64 
 8   Transcription_Count     4062 non-null   int64 
 9   Result_Sheet_Stamped    4062 non-null   int64 
 10  Result_Sheet_Corrected  4062 non-null   int64 
 11  Result_Sheet_Invalid    4062 non-null   int64 
 12  Result_Sheet_Unclear    4062 non-null   int64 
 13  Result_Sheet_Unsigned   4062 non-null   object
 14  APC                     4062 non-null   int64 
 15  LP  

In [40]:
combined_df["PU-Name"]

0             RAILWAY QUARTERS - RAILWAY QUARTERS I
1            RAILWAY QUARTERS - RAILWAY QUARTERS II
2           RAILWAY QUARTERS - RAILWAY QUARTERS III
3                           ABIA POLY - ABIA POLY I
4       LEVER BROTHERS GATE- LEVER BROTHERS GATE IV
                           ...                     
4057                    UMUBAA ALALA VILLAGE SQUARE
4058                      AMAONYEKWERE OSSAH ROAD I
4059               UMUWAYA OBUOHIA VILLAGE HALL III
4060                 AMAONYEKWERE  OSSAH ROAD (III)
4061               UMUOLEGHE AZUAHIA VILLAGE HALL I
Name: PU-Name, Length: 4062, dtype: object

In [32]:
from kaggle_secrets import UserSecretsClient

# Retrieve the API key from Kaggle secrets
secret_label = "OPENCAGE_API_KEY"
api_key = UserSecretsClient().get_secret(secret_label)

In [33]:
def get_coordinates(address):
    url = f'https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}'
    response = requests.get(url).json()
    if response['results']:
        return response['results'][0]['geometry']['lat'], response['results'][0]['geometry']['lng']
    return None, None

combined_df['latitude'], combined_df['longitude'] = zip(*combined_df['PU-Name'].apply(get_coordinates))


In [34]:
# Extract latitude and longitude columns
lat_lon = combined_df[['latitude', 'longitude']].values

# Calculate the distance matrix between all polling units
dist_matrix = distance_matrix(lat_lon, lat_lon)

# Define the radius for neighbours (in kilometers)
radius_km = 1.0

# Create a list to store the outlier scores and neighbour information
results = []

In [36]:
# Function to calculate outlier scores and neighbours
def calculate_outlier_score(row, all_units, party, dist_matrix, radius_km):
    neighbours = all_units[(dist_matrix[row.name] <= radius_km) & (all_units.index != row.name)]
    if not neighbours.empty:
        neighbour_votes = neighbours[party].mean()
        outlier_score = abs(row[party] - neighbour_votes)
        return outlier_score, neighbours['PU-Code'].tolist()
    return 0, []

# Iterate over each polling unit to calculate the outlier scores and neighbours
for party in ['APC', 'LP', 'PDP', 'NNPP']:  # Replace with actual party columns
    outlier_scores = []
    neighbours_list = []
    for index, row in combined_df.iterrows():
        score, neighbours = calculate_outlier_score(row, combined_df, party, dist_matrix, radius_km)
        outlier_scores.append(score)
        neighbours_list.append(neighbours)
    combined_df[f'{party}_outlier_score'] = outlier_scores
    combined_df[f'{party}_neighbours'] = neighbours_list


In [37]:
# Sort the dataset by the outlier scores for each party
sorted_apc = sampled_df.sort_values(by='APC_outlier_score', ascending=False).head(3)
sorted_lp = sampled_df.sort_values(by='LP_outlier_score', ascending=False).head(3)
sorted_pdp = sampled_df.sort_values(by='PDP_outlier_score', ascending=False).head(3)
sorted_nnpp = sampled_df.sort_values(by='NNPP_outlier_score', ascending=False).head(3)

# Display the top 3 outliers for each party
print("Top 3 APC Outliers:")
print(sorted_apc)
print("\nTop 3 LP Outliers:")
print(sorted_lp)
print("\nTop 3 PDP Outliers:")
print(sorted_pdp)
print("\nTop 3 NNPP Outliers:")
print(sorted_nnpp)


Top 3 APC Outliers:
    State            LGA                      Ward       PU-Code  \
405  ABIA      ABA SOUTH             ABA TOWN HALL  01-02-12-002   
0    ABIA      ABA NORTH  ST.EUGENES BY OKIGWE RD.  01-01-05-026   
1    ABIA  UMUAHIA NORTH          UMUAHIA URBAN  I  01-15-05-053   

                              PU-Name  Accredited_Voters  Registered_Voters  \
405  ABA TOWN HALL - ABA TOWN HALL II                 70                750   
0         ST EUGENE PRIMARY SCHOOL IX                 34                 60   
1             SCHOOL OF MIDWIFERY FMC                 17                 46   

     Results_Found  Transcription_Count  Result_Sheet_Stamped  ...  latitude  \
405              1                   -1                     1  ...       NaN   
0                1                   -1                     1  ...       NaN   
1                1                   -1                     0  ...       NaN   

     longitude  APC_outlier_score APC_neighbours  LP_outlier_score  \

In [38]:
with open('outlier_report.txt', 'w') as report:
    report.write("Outlier Detection Report\n")
    report.write("=======================\n\n")
    report.write("Methodology:\n")
    report.write("1. Dataset Preparation: Loaded and combined datasets, added geospatial coordinates.\n")
    report.write("2. Neighbour Identification: Identified neighbours within a 1 km radius using distance matrix.\n")
    report.write("3. Outlier Score Calculation: Calculated outlier scores for each party based on vote deviation from neighbours.\n")
    report.write("4. Sorting and Reporting: Sorted the dataset by outlier scores and identified top outliers.\n\n")

    report.write("Top 3 Outliers:\n")
    for i in range(3):
        for party in ['APC', 'LP', 'PDP', 'NNPP']:
            outlier = sampled_df.sort_values(by=f'{party}_outlier_score', ascending=False).iloc[i]
            report.write(f"Polling Unit: {outlier['PU-Code']} - {outlier['PU-Name']}\n")
            report.write(f"{party} Outlier Score: {outlier[f'{party}_outlier_score']}\n")
            report.write("Neighbours: {}\n".format(", ".join(outlier[f'{party}_neighbours'])))
            report.write("\n")

In [13]:
def get_coordinates(address, api_key):
    url = f'https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}'
    response = requests.get(url).json()
    if response['results']:
        return response['results'][0]['geometry']['lat'], response['results'][0]['geometry']['lng']
    return None, None

# Apply geocoding to get latitude and longitude
combined_df['latitude'], combined_df['longitude'] = zip(*combined_df['PU-Name'].apply(lambda x: get_coordinates(x, api_key)))

# Drop rows with missing coordinates
combined_df.dropna(subset=['latitude', 'longitude'], inplace=True)